In [1]:
using CSV
using DataFrames
using Dates
using Distributions
using Serialization
using LinearAlgebra: diagm
using JSON

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
data_dir = "../data/";
outputdatapath = "../data/";

In [4]:
forecast_date = "2020_12_24";

In [5]:
capacity_data = DataFrame(CSV.File("../data/capacity_hhs.csv"));

In [6]:
SCENARIOS = [:moderate];
BEDTYPES  = [:allbeds, :icu, :acute];

In [7]:
los_dist = (
    icu = Weibull(1.58, 13.32),
    acute = Weibull(1.38, 12.88),
    allbeds = Weibull(1.38, 12.88),
);

In [8]:
start_date = Date(2020, 07, 31);
end_date   = Date(2021, 01, 09);
date_range = collect(start_date : Day(1) : end_date);
T = length(date_range);

In [9]:
hospitals = sort(capacity_data.hospital);
N = length(hospitals);

In [10]:
capacity_names_full = ["Base Capacity"];
capacity_names_abbrev = ["baselinecap"];

In [11]:
function load_capacity(hospitals, bedtype, capacity_levels=[:baseline])
    beds_dict = Dict(row.hospital => Dict(
        "icu" => row.capacity_icu,
        "acute" => row.capacity_acute,
        "allbeds" => row.capacity_allbeds,
    ) for row in eachrow(capacity_data))

    if capacity_levels isa Symbol
        capacity = [beds_dict[h][string(bedtype)] for h in hospitals]
    elseif capacity_levels isa AbstractArray
        capacity = hcat([[beds_dict[h][string(bedtype)] for h in hospitals] for l in capacity_levels]...)
    else
        error("Invalid capacity_levels")
    end

    return capacity
end;

In [12]:
function estimate_admitted(active, los_dist)
    T = length(active)
    
    initial = active[1]
    discharged = initial .* (pdf.(los_dist, 0:T-1))

    L = 1.0 .- cdf.(los_dist, 0:T)

    A = [(t′ ≤ t) ? L[t-t′+1] : 0 for t in 1:T, t′ in 1:T]
    b = [active[t] - (initial - sum(discharged[1:t])) for t in 1:T]
    admitted = A \ b
    
    return admitted
end;

function estimate_active(initial, admitted, los_dist)
    T = length(admitted)

    discharged = initial .* (pdf.(los_dist, 0:T-1))

    L = 1.0 .- cdf.(los_dist, 0:T)

    active = [(
        initial
        - sum(discharged[1:t])
        + sum(L[t-t₁+1] * admitted[t₁] for t₁ in 1:t)
    ) for t in 1:T]
    
    return active
end;

In [13]:
function interpolate_missing(xs::Array{Union{Float64,Missing},2})
    output = Array{Float64,2}(undef, size(xs)...)
    for i in 1:size(xs,1)
        output[i,:] = interpolate_missing(xs[i,:])
    end
    return output
end;

function interpolate_missing(xs::Array{Union{Float64,Missing},1})
    if all(ismissing.(xs))
        return zeros(Float64, length(xs))
    end
    
    xs = deepcopy(xs)
    for i in 1:length(xs)
        if ismissing(xs[i])
            a = findprev(y -> !ismissing(y), xs, i)
            b = findnext(y -> !ismissing(y), xs, i)
            
            a = isnothing(a) ? b : a
            b = isnothing(b) ? a : b
            
            m = (a==b) ? 0 : ((xs[b]-xs[a]) / (b-a))
            xs[i] = (m * (i-a)) + xs[a]
        end
    end
    return xs
end;

In [14]:
hhs_data = DataFrame(CSV.File("../data/hhs_data_2020_12_24.csv"))
hhs_data_dict = Dict((row.hospital,row.date) => row for row in eachrow(hhs_data));

In [15]:
forecast = DataFrame(CSV.File(joinpath(data_dir, "hhs_forecast_$(forecast_date).csv")));

In [16]:
function load_data_hhs(scenario, bedtype)
    @assert(bedtype in [:icu, :acute, :allbeds])
    @assert(scenario in [:optimistic, :moderate, :pessimistic, :catastrophic])

    forecast_dict = Dict((row.hospital, row.date) => (
        admitted = row["admitted_$(scenario)_$(bedtype)"],
    ) for row in eachrow(forecast))
    
    hist_dict = Dict(k => (active = v["active_$(bedtype)"], admitted = v["admissions_$(bedtype)"]) for (k,v) in pairs(hhs_data_dict))

    hist_date_range = sort(intersect(date_range, hhs_data.date))
    forecast_date_range = sort(intersect(date_range, forecast.date))

    hist_date_range_t = [findfirst(date_range .== d) for d in hist_date_range]
    forecast_date_range_t = [findfirst(date_range .== d) for d in forecast_date_range]
    
    hist_active = [haskey(hist_dict,(h,d)) ? hist_dict[(h,d)].active : missing for h in hospitals, d in hist_date_range]
    hist_admitted = [haskey(hist_dict,(h,d)) ? hist_dict[(h,d)].admitted : missing for h in hospitals, d in hist_date_range]
    
    forecast_initial = hist_active[:,end]
    
    forecast_admitted = [haskey(forecast_dict,(h,d)) ? forecast_dict[(h,d)].admitted : missing for h in hospitals, d in forecast_date_range]
    forecast_active   = permutedims(hcat([estimate_active(forecast_initial[i], forecast_admitted[i,:], los_dist.allbeds) for i in 1:N]...), (2,1))

    active = Array{Union{Float64,Missing},2}(undef, N, T)
    fill!(active, missing)
    
    active[:,forecast_date_range_t] = forecast_active
    active[:,hist_date_range_t] = hist_active
    
    admitted = Array{Union{Float64,Missing},2}(undef, N, T)
    fill!(admitted, missing)
    
    admitted[:,forecast_date_range_t] = forecast_admitted
    admitted[:,hist_date_range_t] = hist_admitted
    
    active = interpolate_missing(active)
    admitted = interpolate_missing(admitted)

    admitted_uncertainty = 0.1 .* admitted

    beds = load_capacity(hospitals, bedtype, :baseline)
    capacity = load_capacity(hospitals, bedtype, [:baseline,])

    data = (
        scenario = scenario,
        bedtype = bedtype,

        los_dist = los_dist[bedtype],

        active = active,
        admitted = admitted,
        admitted_uncertainty = admitted_uncertainty,

        beds = beds,
        capacity = capacity,
    )

    return data
end;

In [17]:
maindata = Dict()
for scenario in SCENARIOS, bedtype in BEDTYPES
    @show (scenario, bedtype)
    maindata[(scenario,bedtype)] = load_data_hhs(scenario, bedtype)
end

(scenario, bedtype) = (:moderate, :allbeds)
(scenario, bedtype) = (:moderate, :icu)
(scenario, bedtype) = (:moderate, :acute)


In [18]:
hospital_positions_raw = DataFrame(CSV.File("../data/hhs_locations.csv"))
hospital_positions = Dict(row.hospital => (
    lat  = row.lat,
    long = row.long,
) for row in eachrow(hospital_positions_raw));

In [21]:
hospital_meta_df = DataFrame(CSV.File("../data/hhs_hospital_meta.csv"));
hospital_meta = [(
    name = row.hospital,
    id = row.hospital_id,
    index = findfirst(==(row.hospital), hospitals),
    state = row.state,
    zipcode = row.zip,
    city = row.city,
    county = row.fips_code,
) for row in eachrow(hospital_meta_df)];

In [22]:
completedata = (
    location_names = hospitals,
    location_names_short = hospitals,
    location_meta = hospital_meta,
    start_date = start_date,
    end_date = end_date,
    counties = nothing,
    states = nothing,
    dist_matrix = nothing,
    locations_latlong = hospital_positions,
    casesdata = maindata,
);

In [23]:
serialize(joinpath(outputdatapath, "data_hhs.jlser"), completedata);